In [1]:
from jetbot import ObjectDetector
from jetbot import Camera
from jetbot import Robot
from jetbot import bgr8_to_jpeg
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np
import time

# basic setup
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')
camera = Camera.instance(width=300, height=300, fps=10)
robot = Robot()

In [2]:
width = 300
height = 300

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

def estimate_distance(detection):
    focal_length = 154.0  #need to caliborate your camera to get this
    obj_info = object_info_dict[detection['label']]
    bbox = detection['bbox']  
    obj_height = obj_info[1]
    obj_height_in_img = width * (bbox[3] - bbox[1])        
    dist = obj_height * focal_length / obj_height_in_img
    return dist


In [3]:
is_local_steering_in_progress = False
is_moving_to_target_in_progress = False
is_locking_target_in_progress = False
prev_obstacle_label = -1

is_finished_target_locking = True


target_obj_tall = 1
target_obj_short = 31

obstacle_obj_1 = 64
obstacle_obj_2 = 18

object_info_dict = {1:("person", 65), 31:("short_object", 40), 18:("dog", 6.2), 64:("plant", 12)}

def local_steering(detections):
    # check if local steering is needed based on distance
    global is_local_steering_in_progress
    global prev_obstacle_label
    global is_moving_to_target_in_progress
    is_local_steering_needed = False
    
    obstacle = None
    obstacle_dist = 100000
    
    center_offset = 0
    for det in detections:
        bbox = det['bbox']
        label = det['label']
       
        if(label == obstacle_obj_1 or label == obstacle_obj_2):
            dist = estimate_distance(det)
            print(str(label) + " " + str(dist) + " " + str(height * bbox[3]))
            
            is_local_steering_needed = (height * bbox[3] > height - 30)
            if(label == obstacle_obj_1):
                is_local_steering_needed = dist < 17
            elif(label == obstacle_obj_2):
                is_local_steering_needed = dist < 17
            if(is_local_steering_needed):
                obstacle = det
                obstacle_dist = dist
                
                center = detection_center(det)
                center_offset = center[0]
            break
  

    if(is_local_steering_needed and obstacle is not None):  
        #if seen this before, skip local steering
        if(obstacle['label'] == prev_obstacle_label):
            return 
        is_local_steering_in_progress = True
        is_moving_to_target_in_progress = False
        print("start local steering")
        #local_steering code come here
        
        forward_sleep_time = 1
        
        if(obstacle['label'] == obstacle_obj_1):
            forward_sleep_time = 1.4
        elif(obstacle['label'] == obstacle_obj_2):
            forward_sleep_time = 1.2
            
        robot.stop()
        time.sleep(2)
        
        on_center = center_offset <= 0.125 and center_offset >= -0.125
            
        while(on_center == False):
            print("center_offset " + str(center_offset))
            if center_offset > 0.125:
                print("on right")
                robot.right(0.28)
                time.sleep(0.1)
                robot.stop()
                time.sleep(1)
            elif center_offset < -0.125:
                print("on left")
                robot.left(0.28)
                time.sleep(0.1)
                robot.stop()
                time.sleep(1)
                    
            image = camera.value
            detections = model(image)
            visualize_detections(image, detections[0]) 
            for det in detections[0]:
                bbox = det['bbox']
                label = det['label']
                if(label == obstacle_obj_1 or label == obstacle_obj_2):
                    center = detection_center(det)
                    on_center = center[0] < 0.125 and center[0] > -0.125
                    center_offset = center[0]
                    break
            
            
        robot.set_motors(0.3, 0.305)
        time.sleep(0.2)
                
        robot.left(speed=0.25)
        time.sleep(0.32)
        robot.set_motors(0.3, 0.305)
        time.sleep(forward_sleep_time)

        robot.right(speed=0.25)
        time.sleep(0.32)
        robot.set_motors(0.3, 0.305)
        time.sleep(forward_sleep_time)
        
#         robot.left(speed=0.25)
#         time.sleep(0.28)
#         robot.set_motors(0.3, 0.305)
#         time.sleep(forward_sleep_time)
    
        robot.stop()
        time.sleep(1.0)
        print("end local steering")
        prev_obstacle_label = obstacle['label']
        is_local_steering_in_progress = False
    
    
def visualize_detections(image, detections):

    for det in detections:
        bbox = det['bbox']
        label = det['label']
       
        if(label == target_obj_tall or label == target_obj_short):
            dist = estimate_distance(det)
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 0, 255), 2)
            cv2.putText(image, object_info_dict[label][0] + ":" + "{:.2f}".format(dist), (int(width * bbox[0]), int(height * bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255),2)
        elif(label == obstacle_obj_1 or label == obstacle_obj_2):
            dist = estimate_distance(det)
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 2)
            cv2.putText(image, object_info_dict[label][0] + ":" + "{:.2f}".format(dist), (int(width * bbox[0]), int(height * bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0),2)
#             cv2.putText(image, str(height * bbox[3]) + " " + ":" + "{:.2f}".format(dist) + " " + str(height * bbox[3]), (int(width * bbox[0]), int(height * bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0),2)
        
        else:
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
            cv2.putText(image, str(label), (int(width * bbox[0]), int(height * bbox[1]) + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0),2)
    
    image_widget.value = bgr8_to_jpeg(image)
   
    
def lock_and_move_to_target(detections):
    
    
    global is_finished_target_locking

    global is_locking_target_in_progress
    global is_moving_to_target_in_progress
    
    if(is_locking_target_in_progress == True):
        return
    
    if(is_moving_to_target_in_progress == True):
        return
    
    
    is_locking_target_in_progress = True
    
    
    found_object = False
    
    print("start locking target")
    
    while(found_object == False):

   
        is_finished_target_locking = False
        is_locking_target_in_progress = True
        
        image = camera.value
    
        detections = model(image)
        print("visualize ")
        visualize_detections(image, detections[0])
        center_offset = 0
        for det in detections[0]:
            print("detection ")
            bbox = det['bbox']
            label = det['label']
            if(label == target_obj_tall or label == target_obj_short):
                # move toward the target object
                found_object = True
                print("found target object")
                center = detection_center(det)
                center_offset = center[0]
        
                break
        
        if(found_object == True):
            on_center = center_offset <= 0.125 and center_offset >= -0.125
            
            while(on_center == False):
                print("center_offset " + str(center_offset))
                if center_offset > 0.125:
                    print("on right")
                    robot.right(0.28)
                    time.sleep(0.1)
                    robot.stop()
                    time.sleep(1)
                elif center_offset < -0.125:
                    print("on left")
                    robot.left(0.28)
                    time.sleep(0.1)
                    robot.stop()
                    time.sleep(1)
                    
                image = camera.value
                detections = model(image)
                visualize_detections(image, detections[0]) 
                for det in detections[0]:
                    bbox = det['bbox']
                    label = det['label']
                    if(label == target_obj_tall or label == target_obj_short):
                        center = detection_center(det)
                        on_center = center[0] < 0.125 and center[0] > -0.125
                        center_offset = center[0]
                        break

            if(on_center == True):
                is_moving_to_target_in_progress = True
                robot.set_motors(0.3, 0.31)
            
        
        if(found_object == False):
            robot.left(0.28)
            time.sleep(0.2)
            robot.stop()
            time.sleep(2)

    print("end locking target")
        
    is_locking_target_in_progress = False
        


def is_target_reached(detections):
    # check if the car is close enough to the target
    for det in detections:
        bbox = det['bbox']
        label = det['label']
        if(label == target_obj_tall or label == target_obj_short):
            dist = estimate_distance(det)
            return dist < 20
            
    return False
   

In [ ]:
import ipywidgets.widgets as widgets
image_widget = widgets.Image(format='jpeg', width=300, height=300)

display(widgets.VBox([ image_widget]))

def execute(change):

    if(is_local_steering_in_progress == True or is_locking_target_in_progress == True):
        return
    
    image = change['new']
    
    # compute all detected objects
    detections = model(image)

    
    # visualized target and obstacle objects
    visualize_detections(image, detections[0])

    if(is_target_reached(detections[0])):
        print("target reached!!!")
        camera.unobserve_all()
        robot.stop()
        return
    
    if(is_local_steering_in_progress):
        print("is_local_steering_in_progress")
        return
    
    if(is_locking_target_in_progress):
        print("is_locking_target_in_progress")
        return
    

    image_widget.value = bgr8_to_jpeg(image)
    local_steering(detections[0])
    
    if(is_local_steering_in_progress == False and is_locking_target_in_progress == False and is_moving_to_target_in_progress == False):
        camera.unobserve_all()
        camera.observe(execute, names='value')
        lock_and_move_to_target(detections[0])
    

    
execute({'new': camera.value})

camera.unobserve_all()
camera.observe(execute, names='value')

In [5]:
robot.stop()
camera.unobserve_all()